In [136]:
import sys
sys.path.append("../")
import os
from dotenv import load_dotenv
from smart_open import smart_open
import pandas as pd
import numpy as np
import ingest


## Loss-adjusted Food Availability

### Total

In [6]:
load_dotenv()
s3 = ingest.S3AWS(os.getenv("ACCESS_KEY_ID"),os.getenv("SECRET_ACCESS_KEY"))
df_raw = s3.load_df("s3-bucket-raw-usda", "loss-adjusted-food-availability/calories.xls", "xls", 1)
df_raw.head()

,"Average daily per capita calories from the U.S. food availability, adjusted for spoilage and other waste",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Year,"Meat, eggs, and nuts",Dairy,Fruit,Vegetables,Flour and cereal products**,Added fats and oils and dairy fats*,Sugar and sweeteners (Added),Total
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970,508.895424,250.291001,71.242053,135.44114,409.802437,346.18452,332.539441,2054.396014


In [7]:
df = df_raw.copy()
df.columns = df.iloc[0]
last_index = df[df['Year'] == 2017].index[0] + 1
df = df[4: last_index]
df = df.set_index("Year")
fod_median = df["Added fats and oils and dairy fats*"].median()
total_median = df["Total"].median()
df["Added fats and oils and dairy fats*"] = df["Added fats and oils and dairy fats*"].fillna(fod_median)
df["Total"] = df["Total"].fillna(total_median)
df

,"Meat, eggs, and nuts",Dairy,Fruit,Vegetables,Flour and cereal products**,Added fats and oils and dairy fats*,Sugar and sweeteners (Added),Total
Year,,,,,,,,
1970,508.895424,250.291001,71.242053,135.44114,409.802437,346.184520,332.539441,2054.396014
1971,516.361737,250.838785,73.058372,133.028418,403.733801,341.927818,335.319288,2054.268219
1972,515.380701,248.879915,68.71739,132.679686,399.035074,353.225707,339.185262,2057.103735
1973,486.202095,247.493426,71.771854,132.53015,410.792148,356.921131,340.257027,2045.967830
1974,502.622716,240.660385,72.5649,127.587458,406.397868,349.165931,328.810522,2027.809780
1975,494.111343,239.779865,75.998119,131.566708,415.616719,350.567626,317.701931,2025.342311
1976,516.431127,241.377387,77.274811,132.429937,429.822414,367.480149,334.509681,2099.325505
1977,512.02925,240.129874,77.642956,131.711948,417.396023,353.091598,342.454411,2074.456061
1978,506.419883,238.965042,77.612194,126.79488,427.027106,363.768219,338.537446,2079.124771


In [8]:
bucket_name = "s3-bucket-clean-usda"
obj_key = "loss-adjusted-food-availability-clean/calories_total.xls"
s3.df_to_s3(df, bucket_name, obj_key)

### Percent

In [9]:
df_raw = s3.load_df("s3-bucket-raw-usda", "loss-adjusted-food-availability/calories.xls", "xls", 2)
df_raw.head()

,"Percent of total daily per capita calories from the U.S. food availability, adjusted for spoilage and other waste",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Year,"Meat, eggs, and nuts",Dairy,Fruit,Vegetables,Flour and cereal products,Added fats and oils and dairy fats*,Caloric sweeteners,Total
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970,24.771048,12.183192,3.467786,6.592747,19.947587,16.850915,16.186725,100


In [12]:
df = df_raw.copy()
df.columns = df.iloc[0]
last_index = df[df['Year'] == 2010].index[0] + 1
df = df[4: last_index]
df = df.set_index("Year")
df

,"Meat, eggs, and nuts",Dairy,Fruit,Vegetables,Flour and cereal products,Added fats and oils and dairy fats*,Caloric sweeteners,Total
Year,,,,,,,,
1970,24.771048,12.183192,3.467786,6.592747,19.947587,16.850915,16.186725,100
1971,25.136043,12.210615,3.556418,6.475708,19.653412,16.64475,16.323053,100
1972,25.053705,12.098559,3.340492,6.44983,19.397907,17.171021,16.488486,100
1973,23.763917,12.096643,3.507966,6.477626,20.078133,17.4451,16.630615,100
1974,24.786482,11.867996,3.578487,6.291885,20.041222,17.21887,16.215057,100
1975,24.396436,11.83898,3.752359,6.496023,20.520814,17.309056,15.686333,100
1976,24.59986,11.497854,3.680935,6.308214,20.47431,17.504677,15.93415,100
1977,24.682579,11.575558,3.74281,6.349228,20.120745,17.020924,16.508154,100
1978,24.357359,11.49354,3.732926,6.098474,20.538792,17.496219,16.28269,100


# How much loss during the processing of each food group, per capita.

In [106]:
file = pd.ExcelFile("./Loss-Adjusted Food Availability/Dairy.xls")
sheet_names = file.sheet_names[1:]
sheet_names

['Plain whole milk',
 '2 percent milk',
 '1 percent milk',
 'Skim milk',
 'All plain milk',
 'Whole flavored milk',
 'Lower fat flavored milk',
 'All flavored milk',
 'Buttermilk',
 'Eggnog',
 'Miscellaneous fluid milk',
 'All beverage milks',
 'Yogurt',
 'Cheddar cheese',
 'Other American cheese',
 'All American cheese',
 'Provolone cheese',
 'Romano cheese',
 'Parmesan cheese',
 'Mozarella cheese',
 'Ricotta cheese',
 'Other Italian cheese',
 'All Italian cheese',
 'Swiss cheese',
 'Brick cheese',
 'Muenster cheese',
 'Blue cheese',
 'Other miscellaneous cheese',
 'Total miscellaneous cheese',
 'Total cheese',
 'Regular cottage cheese',
 'Lowfat cottage cheese',
 'Total cottage cheese',
 'Regular ice cream',
 'Lowfat and nonfat ice cream',
 'Other frozen',
 'Frozen dairy products',
 'Evap cond canned whole milk',
 'Evap cond bulk whole milk',
 'Evap cond skim milk',
 'All evaporated condensed milk',
 'Dry whole milk',
 'Nonfat dry milk',
 'Dry buttermilk',
 'Dry milk products',
 'Hal

In [307]:
def get_paths(dirname) -> list[str]:
    all_files = os.listdir(dirname)
    return [f"{dirname}{filename}" for filename in all_files]
def has_numbers(input_str):
    if input_str.startswith("19") or input_str.startswith("20"):
        return True
    return False


In [312]:
key = get_paths("./Loss-Adjusted Food Availability/")[:2][0]
print(key)
dff = pd.read_excel(key, sheet_name=11)
dff.columns = dff.iloc[0]
cols = list(dff.columns)


./Loss-Adjusted Food Availability/meat.xls


In [313]:
indx = [i for i, col in enumerate(cols) if col is np.nan]

if len(indx) > 2:
    new_cols = ["Edible weight", "Other loss", "Loss oz/day", "Loss g/day"]
elif len(indx) == 2:
    new_cols = ["Loss os/day", "Loss g/day"]
for i, col in zip(indx, new_cols):
    cols[i] = col
dff.columns = cols
avail_cal = ""
for col in list(dff.columns):
    if col.startswith("Calories"):
       avail_cal = col 
        
new_col_names = {
    "Year": "year",
    "Primary weight2": "original weight",
    "Edible weight": "edible weight",
    "Total loss, all levels": "total percent loss",
    "Per capita availability adjusted for loss": "loss lbs/year",
    "Loss g/day": "loss g/day",
    avail_cal: "available calories/day"
    
}

dff = dff.rename(columns=new_col_names)
final_cols = ["year", "original weight", "edible weight", "total percent loss", "loss lbs/year", "loss g/day", "available calories/day"]
dff = dff[final_cols]
dff

,year,original weight,edible weight,total percent loss,loss lbs/year,loss g/day,available calories/day
0,Year,Primary weight2,Edible weight,"Total loss, all levels",Per capita availability adjusted for loss,NaN,Calories available daily4
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,-- Lbs/year --,-- Lbs/year --,-- Percent --,-- Lbs/year --,-- G/day --,-- Number --
5,1970,6.9,6.286149,45.337831,3.77169,4.687146,5.268026
6,1971,6.7,6.103542,45.341415,3.662125,4.550988,5.097526
7,1972,7.1,6.468757,45.334448,3.881254,4.823304,5.438527
8,1973,7.4,6.743819,45.320388,4.046291,5.028398,5.745463
9,1974,6.9,6.285574,45.342833,3.771345,4.686717,5.242433


In [192]:
os.chdir(os.path.expanduser('~'))
os.chdir("Classes/CSS497/etl_project/stage/")
os.getcwd()

'/Users/narapadychhuoy/Classes/CSS497/etl_project/stage'

In [372]:
test = ["year", "original weight", "edible weight", "total percent loss", "loss lbs/year", "loss g/day", "available calories/day"]
test.remove("edible weight")
test

['year',
 'original weight',
 'total percent loss',
 'loss lbs/year',
 'loss g/day',
 'available calories/day']

In [373]:
os.system(f"rm -rf {new_dir}")

0

In [374]:
dirname = "Loss-Adjusted Food Availability/"
path_names = get_paths(dirname)
new_dir = dirname.lower().replace(" ", "-")[:-1] + "-clean"
os.mkdir(new_dir)

In [375]:
# each file
for path in path_names:
    if "calories" in path or "serving" in path:
        continue
    # get sheets to process
    file = pd.ExcelFile(path)
    sheet_names = file.sheet_names[1:]
    
    sub_dir = path[path.rfind("/") + 1: path.rfind(".")].lower()
    dir_path = os.path.join(new_dir, sub_dir)
    try:
        os.mkdir(dir_path)
    except OSError as e:
        print(e)

    print(f"=======Read {path}===============")
    # each sheet
    for name in sheet_names:
        # read each sheet in the file
        print(f"- {name}")
        df_raw = pd.read_excel(path, sheet_name=name)
        df = df_raw.copy()
        # use title as key 
        # title is header of each sheet
        filename = df.columns[0].split(":")[0].lower()
        filename = filename.replace(" ", "-").replace("/", "-")

        df.columns = df.iloc[0]
        filter = df['Year'].apply(lambda x: has_numbers(str(x)))
        df = df[filter]               
        
        cols = list(df.columns)
        indx = [i for i, col in enumerate(cols) if col is np.nan]

        if len(indx) == 2:
            new_cols = ["Loss oz/day", "Loss g/day"]
        elif len(indx) == 3:
            new_cols = ["Loss gal/year", "Loss oz/day", "Loss g/day"]
        elif len(indx) == 4:
            new_cols = ["Edible weight", "Other loss", "Loss oz/day", "Loss g/day"]
        elif len(indx) == 5:
            new_cols = ["Edible weight", "Other loss", "Loss gal/year", "Loss oz/day", "Loss g/day"]
        for i, col in zip(indx, new_cols):
            cols[i] = col
            
        df.columns = cols
        avail_cal = ""
        primary_weight = ""
        for col in list(df.columns):
            if str(col).startswith("Calories"):
                avail_cal = col 
        for col in list(df.columns):
            if str(col).startswith("Primary"):
                primary_weight = col 
                
        new_col_names = {
            "Year": "year",
            primary_weight: "original weight",
            "Edible weight": "edible weight",
            "Total loss, all levels": "total percent loss",
            "Per capita availability adjusted for loss": "loss lbs/year",
            "Loss g/day": "loss g/day",
            avail_cal: "available calories/day"
            
        }

        df = df.rename(columns=new_col_names)
            
        final_cols = ["year", "original weight", "edible weight", "total percent loss", "loss lbs/year", "loss g/day", "available calories/day"]

        if name == 'Total grains':
            final_cols.remove("edible weight")
        df = df[final_cols]
        print(f"{dir_path}/{filename}.csv")
        df.to_csv(f"{dir_path}/{filename}.csv", index=False)
        

=======Read Loss-Adjusted Food Availability/meat.xls===============
- Beef
loss-adjusted-food-availability-clean/meat/beef.csv
- Veal
loss-adjusted-food-availability-clean/meat/veal.csv
- Pork
loss-adjusted-food-availability-clean/meat/pork.csv
- Lamb
loss-adjusted-food-availability-clean/meat/lamb.csv
- Red meat
loss-adjusted-food-availability-clean/meat/red-meat.csv
- Chicken
loss-adjusted-food-availability-clean/meat/chicken.csv
- Turkey
loss-adjusted-food-availability-clean/meat/turkey.csv
- Poultry
loss-adjusted-food-availability-clean/meat/poultry.csv
- Fresh and frozen fish
loss-adjusted-food-availability-clean/meat/fresh-and-frozen-fish.csv
- Fresh and frozen shellfish
loss-adjusted-food-availability-clean/meat/fresh-and-frozen-shellfish.csv
- Total Fresh and Frozen Fish
loss-adjusted-food-availability-clean/meat/total-fresh-and-frozen-fish-and-shellfish.csv
- Canned Salmon
loss-adjusted-food-availability-clean/meat/canned-salmon.csv
- Canned Sardines
loss-adjusted-food-availab